In [1]:
!echo $USER | sudo -S systemctl restart nvargus-daemon

[sudo] password for jetson: 

In [2]:
%cd fast-and-furious-with-self-drive-ai/scripts/collision_avoidance/training

/home/jetson/fast-and-furious-with-self-drive-ai/scripts/collision_avoidance/training


# Train the model
- This notebook is use to train the model by using the data we collect from `0_data_collection_3class.ipynb`
- We'll train our image classifier to detect three classes `forward`, `turn left` and `turn right`

In [3]:
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [5]:
data_dir = 'combined_data'  # Directory containing the training data
train_dataset = datasets.ImageFolder(data_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

### Define the neural network

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)  # Three outputs：forward/left/right
model = model.to(device)

In [7]:
criterion = nn.CrossEntropyLoss()   # Three categories => CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Train the neural network
Using the code below we will train the neural network for 10 epochs, saving the best performing model after each epoch. Once that is finished, you should see a file `resnet18_3class.pth` in the Jupyter Lab file browser.

In [8]:
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)  # [batch_size, 3]
        loss = criterion(outputs, labels)  
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [1/30], Loss: 0.3155
Epoch [2/30], Loss: 0.0202
Epoch [3/30], Loss: 0.0057
Epoch [4/30], Loss: 0.0044
Epoch [5/30], Loss: 0.0056
Epoch [6/30], Loss: 0.0052
Epoch [7/30], Loss: 0.0029
Epoch [8/30], Loss: 0.0033
Epoch [9/30], Loss: 0.0030
Epoch [10/30], Loss: 0.0030
Epoch [11/30], Loss: 0.0070
Epoch [12/30], Loss: 0.0027
Epoch [13/30], Loss: 0.0113
Epoch [14/30], Loss: 0.0070
Epoch [15/30], Loss: 0.0067
Epoch [16/30], Loss: 0.0022
Epoch [17/30], Loss: 0.0011
Epoch [18/30], Loss: 0.0028
Epoch [19/30], Loss: 0.0032
Epoch [20/30], Loss: 0.0011
Epoch [21/30], Loss: 0.0009
Epoch [22/30], Loss: 0.0010
Epoch [23/30], Loss: 0.0099
Epoch [24/30], Loss: 0.0082
Epoch [25/30], Loss: 0.0135
Epoch [26/30], Loss: 0.0314
Epoch [27/30], Loss: 0.0129
Epoch [28/30], Loss: 0.0651
Epoch [29/30], Loss: 0.0099
Epoch [30/30], Loss: 0.0046


### Save the model

In [9]:
torch.save(model.state_dict(), 'combined_data_model.pth')
print("Training completed and model saved.")

Training completed and model saved.
